In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import csv


class MyDataset(object):
    '''
    Class to load the dataset
    img_dir = Plots dir
    ground_dir = Ground dir where:
        column 1 = Plot name
        column 2 = Plot ground value
        column 3 = Plot GDD value
    '''
    def __init__(self, img_dir, ground_dir):
        self.img_dir = img_dir
        self.ground_dir = ground_dir
        self.df = self._get_img_df()
        self.ground_arr = self._get_ground_df()
        self.GDD_arr = self._get_GDD_df()
    
    def __len__(self):
        return len(self.df)

    def _get_img_df(self):
        img_names = os.listdir(self.img_dir)
        df = pd.DataFrame(img_names, columns=['Filename'])
        df['Global_ID'] = df.Filename.str[4:-8].astype('int') ###[Global_ID] & [6:-8] to be adjusted
        df['DOY'] = df.Filename.str[-7:-4].astype('int') ###[-7:-4] to be adjusted
        df = pd.pivot_table(df, values = 'Filename', index='Global_ID', columns='DOY', aggfunc='first').reset_index() ###[Global_ID] to be adjusted
        df= df.sort_values('Global_ID')
        return(df)

    def _get_ground_df(self):
        ground_arr =  pd.read_csv(self.ground_dir, usecols=(0,1))
        ground_arr = pd.DataFrame(ground_arr)
        ground_arr = ground_arr.sort_values(ground_arr.columns[0])
        ground_arr = ground_arr.to_numpy()
        return(ground_arr)

    def _get_GDD_df(self):
        GDD_arr = pd.read_csv(self.ground_dir, usecols=(0,3))
        GDD_arr = pd.DataFrame(GDD_arr)
        GDD_arr = GDD_arr.sort_values('Global_ID')
        GDD_arr = GDD_arr.to_numpy()
        return(GDD_arr)

    def _transform(self, x):
        x = np.moveaxis(x, -2, -2) ###[np.moveaxis(x, -2, -2)] to be adjusted. Dim goals: Plot:Flight:R:G:B
        ### Normalization or rescale did NOT change the results.

        # x = x.astype('float32')

        # layer = Normalization()
        # layer.adapt(x)
        # x = layer(x)
        # x = Rescaling(1.0 / 255)(x)
        # input_shape = x.shape[:2]
        # inputs = keras.Input(input_shape[0],input_shape[1])
        # x = Rescaling(1.0 / 255)(inputs)  # Rescale inputs
        # x /= 255.0

        # min = x.min()
        # max = x.max()
        # x -= min
        # x /= (max - min)
        
        return(x)

    def __getitem__(self,imgs_stacked):
        alist = [] 
        # GDDlist = [20.8595, 20.7809, 20.9571, 20.6049, 21.0528, 20.8987, 21.1707, 21.6270, 21.2946]
        # GDDdate = 0

        for column in tqdm(self.df.columns[1:]): 
            print(f"Column name:",column)
            imgs_stack = np.stack([cv2.imread(os.path.join(self.img_dir, img)) for img in self.df[column]])
            #imgs_stack = np.stack([cv2.cvtColor(cv2.imread(os.path.join(self.img_dir, img)), cv2.COLOR_BGR2HSV) for img in self.df[column]]) ### Want to change the color space? [cv2.COLOR_BGR2HSV]

            imgs_stack = self._transform(imgs_stack)
            # imgs_stack =  imgs_stack*GDDlist[GDDdate]

            alist.append([imgs_stack])
            # GDDdate += 1
        imgs_stacked = np.squeeze(np.stack(alist, axis=2))                                                                                         

        return (imgs_stacked) 
        #display(arr)

In [3]:
#%cd /content/drive/MyDrive/Soybean_maturity_Trevisan/
main_dir = '/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2022/HURON_Mat'
img_dir = os.path.join(main_dir,'e._ClipPlots_BN')

ground_dir = os.path.join(main_dir,'d._Ground_notes', 'Ground_Notes_HUR_22_table.csv')


In [ ]:
dataset_arr =  MyDataset(img_dir, ground_dir)[object]
len(dataset_arr)

  0%|          | 0/10 [00:00<?, ?it/s]

Column name: 42
Column name: 71
Column name: 73
Column name: 76
Column name: 79
Column name: 83
Column name: 86
Column name: 89
Column name: 92
Column name: 97


132

In [ ]:
dataset_arr.shape

(132, 10, 256, 64, 3)

In [ ]:
image = dataset_arr[111,5]
image.shape
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (image.min(), image.max()))

Min: 0.000, Max: 254.000


In [ ]:
image

In [6]:
dataset_df =  MyDataset(img_dir, ground_dir).df
dataset_df
#dataset_df.to_csv('C:\\temp_maturity\\UMNSoybeans_Planting_Season_2018\\df.csv')

DOY,Global_ID,42,71,73,76,79,83,86,89,92,97
0,22231001,HUR_22231001_042.png,HUR_22231001_071.png,HUR_22231001_073.png,HUR_22231001_076.png,HUR_22231001_079.png,HUR_22231001_083.png,HUR_22231001_086.png,HUR_22231001_089.png,HUR_22231001_092.png,HUR_22231001_097.png
1,22231002,HUR_22231002_042.png,HUR_22231002_071.png,HUR_22231002_073.png,HUR_22231002_076.png,HUR_22231002_079.png,HUR_22231002_083.png,HUR_22231002_086.png,HUR_22231002_089.png,HUR_22231002_092.png,HUR_22231002_097.png
2,22231003,HUR_22231003_042.png,HUR_22231003_071.png,HUR_22231003_073.png,HUR_22231003_076.png,HUR_22231003_079.png,HUR_22231003_083.png,HUR_22231003_086.png,HUR_22231003_089.png,HUR_22231003_092.png,HUR_22231003_097.png
3,22231004,HUR_22231004_042.png,HUR_22231004_071.png,HUR_22231004_073.png,HUR_22231004_076.png,HUR_22231004_079.png,HUR_22231004_083.png,HUR_22231004_086.png,HUR_22231004_089.png,HUR_22231004_092.png,HUR_22231004_097.png
4,22231005,HUR_22231005_042.png,HUR_22231005_071.png,HUR_22231005_073.png,HUR_22231005_076.png,HUR_22231005_079.png,HUR_22231005_083.png,HUR_22231005_086.png,HUR_22231005_089.png,HUR_22231005_092.png,HUR_22231005_097.png
...,...,...,...,...,...,...,...,...,...,...,...
127,22242026,HUR_22242026_042.png,HUR_22242026_071.png,HUR_22242026_073.png,HUR_22242026_076.png,HUR_22242026_079.png,HUR_22242026_083.png,HUR_22242026_086.png,HUR_22242026_089.png,HUR_22242026_092.png,HUR_22242026_097.png
128,22242027,HUR_22242027_042.png,HUR_22242027_071.png,HUR_22242027_073.png,HUR_22242027_076.png,HUR_22242027_079.png,HUR_22242027_083.png,HUR_22242027_086.png,HUR_22242027_089.png,HUR_22242027_092.png,HUR_22242027_097.png
129,22242028,HUR_22242028_042.png,HUR_22242028_071.png,HUR_22242028_073.png,HUR_22242028_076.png,HUR_22242028_079.png,HUR_22242028_083.png,HUR_22242028_086.png,HUR_22242028_089.png,HUR_22242028_092.png,HUR_22242028_097.png
130,22242029,HUR_22242029_042.png,HUR_22242029_071.png,HUR_22242029_073.png,HUR_22242029_076.png,HUR_22242029_079.png,HUR_22242029_083.png,HUR_22242029_086.png,HUR_22242029_089.png,HUR_22242029_092.png,HUR_22242029_097.png


In [ ]:
dataset_ground =  MyDataset(img_dir, ground_dir).ground_arr
dataset_ground

In [ ]:
dataset_GDD =  MyDataset(img_dir, ground_dir).GDD_arr
dataset_GDD

In [ ]:
# Change the current working directory
#os.chdir('H:\\My Drive\\UMNSoybeans_Planting_Season_2018\\e._npy_data')
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2022\\HURON_Mat\\f._numpy_data')
np.save('2022_HUR_11fly.npy', dataset_arr)  

In [ ]:
# Change the current working directory
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2022\\HURON_Mat\\f._numpy_data')
np.save('2022_HUR_ground_9fly.npy', dataset_ground)  

In [9]:
# Change the current working directory
os.chdir('/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2022/HURON_Mat/f._numpy_data')
np.save('2022_HUR_GDD.npy', dataset_GDD) 